In [ ]:
/*----------------------------------------------------------------------------------
Step 1 - Data Ingestion
 As a Tasty Bytes Data Engineer, we will now leverage the newly permissioned stage
 and file formats to ingest and transform our Truck POS data.
 
 Our in-truck POS systems create two seperate files (order_header and order_detail)
 for each order that is processed. Since our foundational order_id's are generated
 in the order_header files let's prioritize ingesting our header data first.
----------------------------------------------------------------------------------*/

--USE CASE 16: RBAC
    
CREATE ROLE IF NOT EXISTS tasty_data_engineer
    COMMENT = 'data engineer for tasty bytes';

--role heirarchy      
GRANT ROLE tasty_data_engineer TO ROLE tasty_admin;

--examples of privileges given to roles
GRANT USAGE ON DATABASE frostbyte_tasty_bytes TO ROLE tasty_data_engineer;
GRANT USAGE ON ALL SCHEMAS IN DATABASE frostbyte_tasty_bytes TO ROLE tasty_data_engineer;
GRANT ALL ON WAREHOUSE tasty_de_wh TO ROLE tasty_data_engineer;

--privileges can also be granted directly to users
GRANT USAGE ON DATABASE frostbyte_tasty_bytes TO USER sfacchine;




-- to begin, let's once again use the Data Engineer role and warehouse
USE ROLE tasty_data_engineer;
USE WAREHOUSE tasty_de_wh;

-- with our context in place, let's now run a list command now filtering on the CSV Order Header directory
LIST @frostbyte_tasty_bytes.public.s3load/raw_pos/order_header/;
    --> demo tip: point out that the order_header data is partitioned by truck_id which is something we will address later.




-- we have seen the files on the stage, let's now use our CSV file format and
-- query the first 100 rows returned from the order_header files
SELECT $1, $2, $3, $4, $5, $6, $7, $8, $9, $10, $11, $12, $13, $14, $15, $16
FROM @frostbyte_tasty_bytes.public.s3load/raw_pos/order_header/
    (FILE_FORMAT => frostbyte_tasty_bytes.public.csv_ff) 
LIMIT 100;


-- uh oh! there are header rows, let's adjust our file format to handle these before moving on
ALTER FILE FORMAT frostbyte_tasty_bytes.public.csv_ff SET skip_header = 1;


-- now let's create our table DDL 
USE ROLE sysadmin;

--USE CASE 5: Enabling CDC
CREATE OR REPLACE TABLE frostbyte_tasty_bytes.raw_pos.order_header_demo
(
    order_id NUMBER(38,0),
    truck_id NUMBER(38,0),
    location_id FLOAT,
    customer_id NUMBER(38,0),
    discount_id VARCHAR(16777216),
    shift_id NUMBER(38,0),
    shift_start_time TIME(9),
    shift_end_time TIME(9),
    order_channel VARCHAR(16777216),
    order_ts TIMESTAMP_NTZ(9),
    served_ts VARCHAR(16777216),
    order_currency VARCHAR(3),
    order_amount NUMBER(38,4),
    order_tax_amount VARCHAR(16777216),
    order_discount_amount VARCHAR(16777216),
    order_total NUMBER(38,4)
)
CHANGE_TRACKING = TRUE;


-- with the table in place, we are now ready to load our data but first let's make sure we
-- are using our Data Engineering Warehouse.
USE ROLE tasty_data_engineer;
USE WAREHOUSE tasty_de_wh;

-- USE CASE 7: With the power of Snowflakes instant elasticity available, let's scale our warehouse up to 2XL
ALTER WAREHOUSE tasty_de_wh SET warehouse_size = '2x-large';

-- now that our warehouse is scaled, let's kick off our COPY INTO script which will
-- ingest our data from the stage into the table we just created.
COPY INTO frostbyte_tasty_bytes.raw_pos.order_header_demo
FROM @frostbyte_tasty_bytes.public.s3load/raw_pos/order_header/
    FILE_FORMAT = 
        (
            FORMAT_NAME = 'frostbyte_tasty_bytes.public.csv_ff'
            -- SKIP_HEADER = 1 -- this was addressed in our file format update so we won't need it!
        );
        
-- with the load complete, let's scale our warehouse back down for now
ALTER WAREHOUSE tasty_de_wh SET warehouse_size = 'xsmall';

select count(*) from frostbyte_tasty_bytes.raw_pos.order_header_demo;
select * from frostbyte_tasty_bytes.raw_pos.order_header_demo limit 10;

-- using our newly loaded table, let's see how many total orders, trucks and locations we ingested data for
SELECT 
    COUNT(DISTINCT oh.order_id) AS count_orders,
    COUNT(DISTINCT oh.truck_id) AS count_trucks,
    COUNT(DISTINCT oh.location_id) AS count_locations
FROM frostbyte_tasty_bytes.raw_pos.order_header_demo oh;


--Change tracking
SET ts1 = (SELECT CURRENT_TIMESTAMP());

 -- Insert new records with dates from the past week
INSERT INTO ORDER_HEADER_DEMO (
    ORDER_ID, TRUCK_ID, LOCATION_ID, CUSTOMER_ID, DISCOUNT_ID, SHIFT_ID, 
    SHIFT_START_TIME, SHIFT_END_TIME, ORDER_CHANNEL, ORDER_TS, SERVED_TS, 
    ORDER_CURRENCY, ORDER_AMOUNT, ORDER_TAX_AMOUNT, ORDER_DISCOUNT_AMOUNT, ORDER_TOTAL
)
VALUES
(1, 1001, 501, 3001, 201, 401, '08:00:00', '16:00:00', 'Online', '2025-03-07 08:30:00', NULL, 'USD', 100.00, 5.00, 10.00, 95.00),
(2, 1002, 502, 3002, 202, 402, '09:00:00', '17:00:00', 'In-Person', '2025-03-09 09:15:00', NULL, 'USD', 150.00, 7.50, 15.00, 142.50),
(3, 1003, 503, 3003, 203, 403, '10:00:00', '18:00:00', 'App', '2025-03-11 10:45:00', NULL, 'USD', 200.00, 10.00, 20.00, 190.00);

-- Delete a record
DELETE FROM ORDER_HEADER_DEMO WHERE ORDER_ID = 1;

-- Update a record
UPDATE ORDER_HEADER_DEMO
SET ORDER_AMOUNT = 175.00, ORDER_TOTAL = 167.50 
WHERE ORDER_ID = 2;


 -- Query the change tracking metadata in the table during the interval from $ts1 to the current time.
 -- Return the full delta of the changes.
SELECT *
 FROM order_header_demo
   CHANGES(INFORMATION => DEFAULT)
   AT(TIMESTAMP => $ts1);

SELECT *
 FROM order_header_demo
   CHANGES(INFORMATION => APPEND_ONLY)
   AT(TIMESTAMP => $ts1);

